In [1]:
from bokeh.plotting import figure, save
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper
import geopandas as gpd
from bokeh.palettes import Viridis6 as palette
import geopandas as gpd
import pysal as ps
import pandas as pd
import numpy as np
import math


In [2]:
grid_fp = r"C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/zips_sea/shp.shp"
network_fp = r"C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/bus_seattle/network.shp"
stops =pd.read_csv('C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/stops-zip.csv')
zips_sea =pd.read_csv('C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/zips_seattle.csv')
zip_route =pd.read_csv('C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/test.csv')

In [3]:
grid = gpd.read_file(grid_fp)
network = gpd.read_file(network_fp)
zip_route = zip_route.dropna(axis=0, how='any')

In [4]:
routes = {}
name = ()
f = 0
for c in range(0,30):
    name = zips_sea.zip[c].astype(str)
    #print(name)
    routes[c] = zip_route['x'][f:(zips_sea['count'][c].astype(int))+f].astype(int).astype(str)
    f = f + zips_sea['count'][c].astype(int)             

In [5]:
network = network.loc[network['ROUTE_NUM'].astype(str) != 'None']
network = network.loc[network['ROUTE_NUM'].astype(str) != '???']

In [6]:
def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]
    
def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )


grid['x'] = grid.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
grid['y'] = grid.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


In [7]:
network0 = network.loc[network.ROUTE_NUM.isin(routes[0]) , :]
network0['zip'] = zips_sea.zip[0]
ns0 = GeoJSONDataSource(geojson=network0.to_json())

network1 = network.loc[network.ROUTE_NUM.isin(routes[1]) , :]
network1['zip'] = zips_sea.zip[1]
ns1 = GeoJSONDataSource(geojson=network1.to_json())

network2 = network.loc[network.ROUTE_NUM.isin(routes[2]) , :]
network2['zip'] = zips_sea.zip[2]
ns2 = GeoJSONDataSource(geojson=network2.to_json())

network3 = network.loc[network.ROUTE_NUM.isin(routes[3]) , :]
network3['zip'] = zips_sea.zip[3]
ns3 = GeoJSONDataSource(geojson=network3.to_json())

network4 = network.loc[network.ROUTE_NUM.isin(routes[4]) , :]
network4['zip'] = zips_sea.zip[4]
ns4 = GeoJSONDataSource(geojson=network4.to_json())

network5 = network.loc[network.ROUTE_NUM.isin(routes[5]) , :]
network5['zip'] = zips_sea.zip[5]
ns5 = GeoJSONDataSource(geojson=network5.to_json())

network6 = network.loc[network.ROUTE_NUM.isin(routes[6]) , :]
network6['zip'] = zips_sea.zip[6]
ns6 = GeoJSONDataSource(geojson=network6.to_json())

network7 = network.loc[network.ROUTE_NUM.isin(routes[7]) , :]
network7['zip'] = zips_sea.zip[7]
ns7 = GeoJSONDataSource(geojson=network7.to_json())

network8 = network.loc[network.ROUTE_NUM.isin(routes[8]) , :]
network8['zip'] = zips_sea.zip[8]
ns8 = GeoJSONDataSource(geojson=network8.to_json())

network9 = network.loc[network.ROUTE_NUM.isin(routes[9]) , :]
network9['zip'] = zips_sea.zip[9]
ns9 = GeoJSONDataSource(geojson=network9.to_json())

network10 = network.loc[network.ROUTE_NUM.isin(routes[10]) , :]
network10['zip'] = zips_sea.zip[10]
ns10 = GeoJSONDataSource(geojson=network10.to_json())

network11 = network.loc[network.ROUTE_NUM.isin(routes[11]) , :]
network11['zip'] = zips_sea.zip[11]
ns11 = GeoJSONDataSource(geojson=network11.to_json())

network12 = network.loc[network.ROUTE_NUM.isin(routes[12]) , :]
network12['zip'] = zips_sea.zip[12]
ns12 = GeoJSONDataSource(geojson=network12.to_json())

network13 = network.loc[network.ROUTE_NUM.isin(routes[13]) , :]
network13['zip'] = zips_sea.zip[13]
ns13 = GeoJSONDataSource(geojson=network13.to_json())

network14 = network.loc[network.ROUTE_NUM.isin(routes[14]) , :]
network14['zip'] = zips_sea.zip[14]
ns14 = GeoJSONDataSource(geojson=network14.to_json())

network15 = network.loc[network.ROUTE_NUM.isin(routes[15]) , :]
network15['zip'] = zips_sea.zip[15]
ns15 = GeoJSONDataSource(geojson=network15.to_json())

network16 = network.loc[network.ROUTE_NUM.isin(routes[16]) , :]
network16['zip'] = zips_sea.zip[16]
ns16 = GeoJSONDataSource(geojson=network16.to_json())

network17 = network.loc[network.ROUTE_NUM.isin(routes[17]) , :]
network17['zip'] = zips_sea.zip[17]
ns17 = GeoJSONDataSource(geojson=network17.to_json())

network18 = network.loc[network.ROUTE_NUM.isin(routes[18]) , :]
network18['zip'] = zips_sea.zip[18]
ns18 = GeoJSONDataSource(geojson=network18.to_json())

network19 = network.loc[network.ROUTE_NUM.isin(routes[19]) , :]
network19['zip'] = zips_sea.zip[19]
ns19 = GeoJSONDataSource(geojson=network19.to_json())

network20 = network.loc[network.ROUTE_NUM.isin(routes[20]) , :]
network20['zip'] = zips_sea.zip[20]
ns20 = GeoJSONDataSource(geojson=network20.to_json())

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [8]:
breaks = [x for x in range(55000, 110000, 5000)]
breaks


#Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
pt_classif = grid[['income']].apply(classifier)


C:\Users\ASUS\Anaconda3\lib\site-packages\pysal\esda\mapclassify.py:267: RuntimeWarning: invalid value encountered in greater
  binIds += (x > l) * (x <= r) * k
C:\Users\ASUS\Anaconda3\lib\site-packages\pysal\esda\mapclassify.py:267: RuntimeWarning: invalid value encountered in less_equal
  binIds += (x > l) * (x <= r) * k
C:\Users\ASUS\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [9]:
# Rename the classified column
pt_classif.columns = ['incomeb']

# Join it back to the grid layer
grid = grid.join(pt_classif)

In [10]:
grid['bin']= pd.np.where(grid.incomeb.astype(str) == '1', "[55000-60000]",pd.np.where(grid.incomeb.astype(str) == '2', "[60000-65000]",
                                                                                      pd.np.where(grid.incomeb.astype(str) == '3', "[65000-70000]",
                                                                                                 pd.np.where(grid.incomeb.astype(str) == '4', "[70000-75000]",
                                                                                                            pd.np.where(grid.incomeb.astype(str) == '5', "[75000-80000]",
                                                                                                                       pd.np.where(grid.incomeb.astype(str) == '6', "[80000-85000]",
                                                                                                                                  pd.np.where(grid.incomeb.astype(str) == '7', "[85000-90000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '8', "[90000-95000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '9', "[95000-100000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '10', "[100000-105000]",
                                                                                                                                             pd.np.where(grid.incomeb.astype(str) == '11', "[105000-110000]",'NA')
                        ))))))))))

In [11]:
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

In [12]:
from bokeh.palettes import Viridis11 as palette
from bokeh.models import LogColorMapper
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CustomJS 
from bokeh.plotting import figure, output_file, show 
from bokeh.events import Tap 
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button,CheckboxGroup
from bokeh.layouts import column, row,gridplot

In [16]:
TOOLS = "pan,wheel_zoom,reset,poly_select,box_select,tap,box_zoom"
p = figure(title="Seattle Bus Routes",tools=TOOLS)

# Plot grid
grid2 = p.patches('x', 'y', source=gsource,
         fill_color='blue',
         fill_alpha=0.010, line_color="black", line_width=1)


#p.legend.location = "top_right"
#p.legend.orientation = "vertical"

r0=p.multi_line('xs', 'ys', source=ns0, color="green", line_width=0.5)
r1=p.multi_line('xs', 'ys', source=ns1, color="green", line_width=0.5)
r2=p.multi_line('xs', 'ys', source=ns2, line_width=0.5, color='green')
r3=p.multi_line('xs', 'ys', source=ns3, color="green", line_width=0.5)
r4=p.multi_line('xs', 'ys', source=ns4, line_width=0.5, color='green')
r5=p.multi_line('xs', 'ys', source=ns5, color="green", line_width=0.5)
r6=p.multi_line('xs', 'ys', source=ns6, line_width=0.5, color='green')
r7=p.multi_line('xs', 'ys', source=ns7, color="green", line_width=0.5)
r8=p.multi_line('xs', 'ys', source=ns8, line_width=0.5, color='green')
r9=p.multi_line('xs', 'ys', source=ns9, color="green", line_width=0.5)
r10=p.multi_line('xs', 'ys', source=ns10, line_width=0.5, color='green')
r11=p.multi_line('xs', 'ys', source=ns11, color="green", line_width=0.5)
r12=p.multi_line('xs', 'ys', source=ns12, line_width=0.5, color='green')
r13=p.multi_line('xs', 'ys', source=ns13, color="green", line_width=0.5)
r14=p.multi_line('xs', 'ys', source=ns14, line_width=0.5, color='green')
r15=p.multi_line('xs', 'ys', source=ns15, color="green", line_width=0.5)
r16=p.multi_line('xs', 'ys', source=ns16, line_width=0.5, color='green')
r17=p.multi_line('xs', 'ys', source=ns17, color="green", line_width=0.5)
r18=p.multi_line('xs', 'ys', source=ns18, line_width=0.5, color='green')
r19=p.multi_line('xs', 'ys', source=ns19, color="green", line_width=0.5)
r20=p.multi_line('xs', 'ys', source=ns20, line_width=0.5, color='green')



#ghover = HoverTool(renderers=[r])
#ghover.tooltips=[("Bus Number", "@ROUTE_NUM")]

ghover = HoverTool(renderers=[grid2])
ghover.tooltips=[("zip code", "@GEOID10")]


#p.circle(x='stop_lon', y='stop_lat', size=1, color="green",source=stops)

def callback(attr, old, new):

    # The index of the selected glyph is : new['1d']['indices'][0]
    patch_name =  gsource.data['GEOID10'][new['1d']['indices'][0]]
    print("TapTool callback executed on Patch {}".format(patch_name))

grid2.data_source.on_change('selected',callback)

curdoc().add_root(column(p))

p.add_tools(ghover)

#outfp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Docs/map.html"
#save(p, outfp)

TOOLS = "pan,wheel_zoom,reset,poly_select,box_select,tap"

color_mapper = LogColorMapper(palette=palette)
p2 = figure(title="PSRC Income",tools=TOOLS)
p2.patches('x', 'y', source=gsource,
         fill_color={'field': 'incomeb', 'transform' : color_mapper},
         fill_alpha=10, line_color="black", line_width=1, legend = 'bin')

p2.legend.location = "top_right"
p2.legend.orientation = "vertical"

checkbox = CheckboxGroup(labels=list(zips_sea['zip'][0:20].astype(str)), active= list(range(0,20)))

checkbox.callback = CustomJS(args=dict(l0=r0, l1=r1, l2=r2, l3=r3,l4=r4, l5=r5, l6=r6, l7=r7, l8=r8,l9=r9,l10=r10,l11=r11,
                               l12=r12,l13=r13,l14=r14,l15=r15,l16=r16,l17=r17,l18=r18,l19=r19,l20=r20     ), 
                             code="""
    //console.log(cb_obj.active);
    l0.visible = false;
    l1.visible = false;
    l2.visible = false;
    l3.visible = false;
    l4.visible = false;
    l5.visible = false;
    l6.visible = false;
    l7.visible = false;
    l8.visible = false;
    l9.visible = false;
    l10.visible = false;
    l11.visible = false;
    l12.visible = false;
    l13.visible = false;
    l14.visible = false;
    l15.visible = false;
    l16.visible = false;
    l17.visible = false;
    l18.visible = false;
    l19.visible = false;
    l20.visible = false;

    for (i in cb_obj.active) {
        //console.log(cb_obj.active[i]);
        if (cb_obj.active[i] == 0) {
            l0.visible = true;
        } else if (cb_obj.active[i] == 1) {
            l1.visible = true;
        } else if (cb_obj.active[i] == 2) {
            l2.visible = true;
        } else if (cb_obj.active[i] == 3) {
            l3.visible = true;
        } else if (cb_obj.active[i] == 4) {
            l4.visible = true;
        } else if (cb_obj.active[i] == 5) {
            l5.visible = true;
        } else if (cb_obj.active[i] == 6) {
            l6.visible = true;
        } else if (cb_obj.active[i] == 7) {
            l7.visible = true;
        } else if (cb_obj.active[i] == 8) {
            l8.visible = true;
        } else if (cb_obj.active[i] == 9) {
            l9.visible = true;
        } else if (cb_obj.active[i] == 10) {
            l10.visible = true;
        } else if (cb_obj.active[i] == 11) {
            l11.visible = true;
        } else if (cb_obj.active[i] == 12) {
            l12.visible = true;
        } else if (cb_obj.active[i] == 13) {
            l13.visible = true;
        } else if (cb_obj.active[i] == 14) {
            l14.visible = true;
        } else if (cb_obj.active[i] == 15) {
            l15.visible = true;
        } else if (cb_obj.active[i] == 16) {
            l16.visible = true;
        } else if (cb_obj.active[i] == 17) {
            l17.visible = true;
        } else if (cb_obj.active[i] == 18) {
            l18.visible = true;
        } else if (cb_obj.active[i] == 19) {
            l19.visible = true;
        } else if (cb_obj.active[i] == 20) {
            l20.visible = true;

        }
    }
""")
checkbox_code = """
    //console.log(cb_obj.active);
    l0.visible = false;
    l1.visible = false;
    l2.visible = false;
    l3.visible = false;
    l4.visible = false;
    l5.visible = false;
    l6.visible = false;
    l7.visible = false;
    l8.visible = false;
    l9.visible = false;
    l10.visible = false;
    l11.visible = false;
    l12.visible = false;
    l13.visible = false;
    l14.visible = false;
    l15.visible = false;
    l16.visible = false;
    l17.visible = false;
    l18.visible = false;
    l19.visible = false;
    l20.visible = false;

    for (i in cb_obj.active) {
        //console.log(cb_obj.active[i]);
        if (cb_obj.active[i] == 0) {
            l0.visible = true;
        } else if (cb_obj.active[i] == 1) {
            l1.visible = true;
        } else if (cb_obj.active[i] == 2) {
            l2.visible = true;
        } else if (cb_obj.active[i] == 3) {
            l3.visible = true;
        } else if (cb_obj.active[i] == 4) {
            l4.visible = true;
        } else if (cb_obj.active[i] == 5) {
            l5.visible = true;
        } else if (cb_obj.active[i] == 6) {
            l6.visible = true;
        } else if (cb_obj.active[i] == 7) {
            l7.visible = true;
        } else if (cb_obj.active[i] == 8) {
            l8.visible = true;
        } else if (cb_obj.active[i] == 9) {
            l9.visible = true;
        } else if (cb_obj.active[i] == 10) {
            l10.visible = true;
        } else if (cb_obj.active[i] == 11) {
            l11.visible = true;
        } else if (cb_obj.active[i] == 12) {
            l12.visible = true;
        } else if (cb_obj.active[i] == 13) {
            l13.visible = true;
        } else if (cb_obj.active[i] == 14) {
            l14.visible = true;
        } else if (cb_obj.active[i] == 15) {
            l15.visible = true;
        } else if (cb_obj.active[i] == 16) {
            l16.visible = true;
        } else if (cb_obj.active[i] == 17) {
            l17.visible = true;
        } else if (cb_obj.active[i] == 18) {
            l18.visible = true;
        } else if (cb_obj.active[i] == 19) {
            l19.visible = true;
        } else if (cb_obj.active[i] == 20) {
            l20.visible = true;

        }
    }
"""
##widget_box_1 = widgetbox(checkbox)
#layout = row(p, widgetbox(checkbox))
myplots = gridplot([[p2,p]])
#p2 = figure(title="PSRC Income",tools=TOOLS)

#p2 = p2.patches('x', 'y', source=gsource,
     #    fill_color={'field': 'income2', 'transform' : color_mapper},
       #  fill_alpha=0.010, line_color="black", line_width=1)

clear_button = Button(label='Clear all',width=200)
clear_button_code = """checkbox.set("active",[]);"""+checkbox_code
clear_button.callback = CustomJS(args=dict(l0=r0, l1=r1, l2=r2, l3=r3,l4=r4, l5=r5, l6=r6, l7=r7, l8=r8,l9=r9,l10=r10,l11=r11,
                               l12=r12,l13=r13,l14=r14,l15=r15,l16=r16,l17=r17,l18=r18,l19=r19,l20=r20), code=clear_button_code)

#check_button = Button(label='Check all',width=200)
#check_button_code = """checkbox.set("active","""+str(N_plots)+""");"""+checkbox_code
#check_button.callback = CustomJS(args=dict(l0=r0, l1=r1, l2=r2, l3=r3,l4=r4, l5=r5, l6=r6, l7=r7, l8=r8,l9=r9,l10=r10,l11=r11,
 #                              l12=r12,l13=r13,l14=r14,l15=r15,l16=r16,l17=r17,l18=r18,l19=r19,l20=r20, lang="coffeescript", code=check_button_code)

#group = widgetbox(checkbox)
        

layout = row(myplots, widgetbox(checkbox,clear_button))

outfp = r"C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/examples/map.html"
output_file(filename, title='Bokeh Plot', mode='cdn', root_dir=None)[source]
#save(layout, outfp)
show(layout)


In [ ]:
color_mapper
